## Vehicle Detector (COCO)

In [1]:
from ultralytics import YOLO
import cv2

In [2]:
# Load YOLOv8 model
model = YOLO("yolov8s.pt")

# COCO class names
class_list = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train',
    'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter',
    'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
    'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase',
    'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
    'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle',
    'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut',
    'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet',
    'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
    'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
    'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

# Vehicle classes we care about
VEHICLE_CLASSES = {
    'car',
    'motorcycle',
    'bus',
    'truck'   # includes vans & lorries in COCO
}

In [3]:
# Load image
image_path = "test_image.jpg"
image = cv2.imread(image_path)

# Run YOLO inference
results = model.predict(image, conf=0.4)

# Extract detections
detections = results[0].boxes.data.detach().cpu().numpy()

WARNING ⚠️ 'source' is missing. Using 'source=/home/joshu/miniconda3/envs/py312/lib/python3.12/site-packages/ultralytics/assets'.


[ WARN:0@7.988] global loadsave.cpp:275 findDecoder imread_('test_image.jpg'): can't open/read file: check file path/integrity



image 1/2 /home/joshu/miniconda3/envs/py312/lib/python3.12/site-packages/ultralytics/assets/bus.jpg: 640x480 4 persons, 1 bus, 562.0ms
image 2/2 /home/joshu/miniconda3/envs/py312/lib/python3.12/site-packages/ultralytics/assets/zidane.jpg: 384x640 2 persons, 1 tie, 548.4ms
Speed: 12.2ms preprocess, 555.2ms inference, 13.4ms postprocess per image at shape (1, 3, 384, 640)


In [4]:
vehicle_detections = []

for det in detections:
    x1, y1, x2, y2 = map(int, det[:4])
    confidence = float(det[4])
    class_id = int(det[5])
    class_name = class_list[class_id]

    if class_name in VEHICLE_CLASSES:
        vehicle_detections.append({
            "class": class_name,
            "confidence": confidence,
            "bbox": (x1, y1, x2, y2)
        })

        # Draw bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"{class_name} {confidence:.2f}"
        cv2.putText(image, label, (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

In [10]:
MAX_WIDTH = 1200  # change if needed

h, w = image.shape[:2]
scale = MAX_WIDTH / w

if scale < 1:
    display_image = cv2.resize(
        image,
        (int(w * scale), int(h * scale))
    )
else:
    display_image = image

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# Show result
cv2.imshow("Vehicle Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()